# Select patients matching certain criteria from the gaitbase
Read all the patients from the gaitbase database, select patients matching certain criteria (e.g. age, diagnois, etc.), and save them to an excel file

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import sqlite3

In [2]:
DB_FILE_NAME = '/home/andrey/scratch/gaitbase/patients.db'
OUT_FNAME = '/home/andrey/scratch/gaitbase/patients.xlsx'
CODE_TYPES = ['H', 'C', 'D']

# Select the patients who are AGE_MIN years old or older at REF_DATE_MIN
# and AGE_MAX years old or younger at REF_DATE_MAX
REF_DATE_MIN = '120625'
REF_DATE_MAX = '010925'
AGE_MIN = 6
AGE_MAX = 15

In [3]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(DB_FILE_NAME)
df = pd.read_sql_query("SELECT * from patients", con)
con.close()

In [4]:
def comp_age_full_years(birth_date, ref_date):
    """Calculate the age in full years at a given reference date."""
    b_date = datetime.strptime(birth_date, '%d%m%y')
    ref_date = datetime.strptime(ref_date, '%d%m%y')

    age = relativedelta(ref_date, b_date).years
    return age

In [5]:

def selector(row):
    if not (row.patient_code[0] in CODE_TYPES):
        return False
    
    b_date = datetime.strptime(row.ssn[:6], '%d%m%y')
    ref_date_min = datetime.strptime(REF_DATE_MIN, '%d%m%y')
    ref_date_max = datetime.strptime(REF_DATE_MAX, '%d%m%y')

    age_min = relativedelta(ref_date_min, b_date).years
    age_max = relativedelta(ref_date_max, b_date).years

    return (age_max < AGE_MAX) and (age_min >= AGE_MIN)

In [6]:
df_filtered = df.loc[df.apply(selector, axis=1)]

df_filtered = df_filtered.copy()  # Create a copy to avoid the warning
df_filtered[f'age (full years) at {REF_DATE_MIN}'] = df_filtered['ssn'].apply(lambda x: comp_age_full_years(x[:6], REF_DATE_MIN))
df_filtered[f'age (full years) at {REF_DATE_MAX}'] = df_filtered['ssn'].apply(lambda x: comp_age_full_years(x[:6], REF_DATE_MAX))

df_filtered[['firstname', 'lastname', 'ssn', 'patient_code', 'diagnosis', f'age (full years) at {REF_DATE_MIN}', f'age (full years) at {REF_DATE_MAX}']].to_excel(OUT_FNAME, index=False)